 ### 'PANDAS'
  'Pandas' es un estándar de la industria para analizar datos en Python. Con unas pocas pulsaciones de teclas, puedes cargar, filtrar, reestructurar y visualizar gigabytes de información heterogénea. Está desarrollado sobre la biblioteca 'NumPy' y toma prestados muchos de sus conceptos y convenciones de sintaxis, por lo que si te sientes cómodo con 'NumPy', Pandas te resultará una herramienta bastante familiar. E incluso si nunca has oído hablar de 'NumPy', Pandas ofrece una gran oportunidad para resolver problemas de análisis de datos con poca o ninguna experiencia en programación.

Dos características clave que Pandas aporta a las matrices 'NumPy' son:

1. Tipos heterogéneos: cada columna puede tener su propio tipo;

2. Índice: mejora la velocidad de búsqueda de las columnas especificadas.

Resulta que estas características son suficientes para hacer de 'Pandas' un poderoso competidor tanto de las 'hojas de cálculo' como de las 'bases de datos'.

'Polars', la reciente reencarnación de Pandas (escrita en 'Rust', por lo tanto más rápida ¹) ya no usa 'NumPy' bajo el capó, pero la sintaxis es bastante similar, por lo que aprender 'Pandas' también te permitirá sentirte cómodo con 'Polars'.

<img src="Img/Articulo_2_diagram_1.png">

## Parte 1. Motivación y presentación

Supongamos que tienes un archivo con un millón de líneas de valores separados por comas como este:

<img src="Img/Articulo_2_diagram_2.png">

Y necesitas dar respuestas a preguntas básicas como "¿Qué ciudades tienen un área de más de 450 km² y una población de menos de 10 millones" con 'NumPy'.

La solución de fuerza bruta de introducir toda la tabla en una matriz, 'NumPy' no es una buena opción: normalmente, las matrices 'NumPy' son homogéneas (todos los valores tienen el mismo tipo), por lo que todos los campos se interpretarán como 'cadenas' y las comparaciones no funcionarán como se espera.

Sí, 'NumPy' tiene matrices estructuradas y de registros que permiten columnas de distintos tipos, pero están pensadas principalmente para interactuar con código 'C'. Cuando se utilizan para fines generales, tienen las siguientes desventajas:

- No es realmente intuitivo (por ejemplo, te enfrentarás a constantes como <f8 y <U8 en todas partes);

- Tiene algunos problemas de rendimiento en comparación con las matrices 'NumPy' normales;

- Se almacenan de forma contigua en la memoria, por lo que cada adición o eliminación de columna requiere la reasignación de toda la matriz;

- Todavía falta mucha funcionalidad de Pandas DataFrames.

Tu próximo intento probablemente sería almacenar cada columna como un vector 'NumPy' independiente. Y después de eso, tal vez envolverlas en un vector 'dict' para que sea más fácil restaurar la integridad de la "base de datos" si decides agregar o eliminar una fila o dos más adelante. Así es como se vería:

<img src="Img/Articulo_2_diagram_3.png">

Si lo has hecho, ¡felicitaciones! Has dado el primer paso para reimplementar Pandas. 

Ahora, aquí hay un par de ejemplos de lo que Pandas puede hacer por usted y que 'NumPy' no puede (o requiere un esfuerzo significativo para lograr).

### Exhibición de Pandas

Considere la siguiente tabla:

<img src="Img/Articulo_2_diagram_4.png">

Describe la variada línea de productos de una 'tienda online' con un total de cuatro productos distintos. A diferencia del ejemplo anterior, se puede representar con una matriz 'NumPy' o con un 'DataFrame de Pandas' igualmente bien. Pero veamos algunas operaciones comunes con él.

#### 1. Clasificación

La ordenación por columna es más legible con Pandas, como puedes ver a continuación:

<img src="Img/Articulo_2_diagram_5.png">

Aquí se calcula 'argsort(a[:,1])', la permutación que hace que la segunda columna de 'a' se ordene en orden ascendente y luego la externa 'a[…]' reordena las filas de 'a', según corresponda. Pandas puede hacerlo en un solo paso.

#### 2. Ordenar por varias columnas

Si necesitamos ordenar por columna de 'price' y desempatar usando la columna de 'weight', la situación empeora para 'NumPy':

<img src="Img/Articulo_2_diag_6.png">

Con 'NumPy', primero ordenamos por 'price' y luego aplicamos un segundo ordenamiento por 'weight'. Un algoritmo de ordenamiento estable garantiza que el resultado del primer ordenamiento no se pierda durante el segundo. Hay otras formas de hacerlo con 'NumPy', pero ninguna es tan simple y elegante como con Pandas.

#### 3. Agregar una columna

Agregar columnas es mucho mejor con Pandas, sintácticamente y arquitectónicamente:

<img src="Img/Articulo_2_diag_7.png">

'Pandas' no necesita reasignar memoria para toda la matriz como 'NumPy'; simplemente agrega una referencia a una nueva columna y actualiza un "registro" de los nombres de las columnas.

#### 4. Búsqueda rápida de elementos

Con las matrices 'NumPy', incluso si el elemento que busca es el primero, necesitará un tiempo proporcional al tamaño de la matriz para encontrarlo. Con 'Pandas', puede indexar las columnas que espera que se consulten con más frecuencia y reducir el tiempo de búsqueda a una constante.

<img src="Img/Articulo_2_diag_8.png">

La columna de índice tiene las siguientes limitaciones:

- Requiere memoria y tiempo para construirse.

- Es de sólo lectura (debe reconstruirse después de cada operación de adición o eliminación).

- No es necesario que los valores sean únicos, pero la aceleración solo ocurre cuando los elementos son únicos.

- Requiere calentamiento: la primera consulta es algo más lenta que en 'NumPy', pero las siguientes son significativamente más rápidas.

#### 5. Uniones por columnas

Si desea complementar una tabla con información de otra tabla basada en una columna común, 'NumPy' no es de mucha ayuda. 'Pandas' es mejor, especialmente para relaciones '1:n'.

<img src="Img/Articulo_2_diag_9.png">

En 'Pandas', 'join' tiene todos los modos de unión familiares: 'inner', 'left', 'right' y 'outer'.

#### 6. Agrupación por columnas

Otra operación habitual en el análisis de datos es la agrupación por columnas. Por ejemplo, para obtener la cantidad total vendida de cada producto, puede hacer lo siguiente:

<img src="Img/Articulo_2_diag_10.png">

Además de 'sum', Pandas admite todo tipo de funciones agregadas: 'mean', 'max', 'min', 'count', etc.

#### 7. Tablas dinámicas
Una de las funciones más potentes de Pandas es una tabla “pivotante”. Es algo así como proyectar un espacio multidimensional en un plano bidimensional.

<img src="Img/Articulo_2_diag_11.png">

Si bien es posible implementarlo con 'NumPy', esta funcionalidad falta "de fábrica", aunque está presente en todas las principales bases de datos relacionales y aplicaciones de hojas de cálculo ( Excel , Google Sheets ).

Pandas también tiene una función 'df.pivot_table' que combina agrupación y pivoteo en una sola herramienta.

En este punto, es posible que te preguntes por qué alguien usaría 'NumPy' si Pandas es tan bueno. 'NumPy' no es mejor ni peor, solo tiene diferentes casos de uso:

- Números aleatorios (por ejemplo, para pruebas)

- Álgebra lineal (por ejemplo, para redes neuronales)

- Imágenes y pilas de imágenes (por ejemplo, para CNN)

- Personal científico de diferenciación, integración, trigonometría y otros.

En pocas palabras, las dos diferencias principales entre NumPy y Pandas son las siguientes:

<img src="Img/Articulo_2_diag_12.png">

Ahora veamos si esas características se consiguen a costa de una pérdida de rendimiento.

### La velocidad de los pandas

He evaluado 'NumPy' y 'Pandas' en una carga de trabajo típica de Pandas: 5 a 100 columnas; 10³ a 10⁸ filas; números enteros y flotantes. Estos son los resultados para 1 fila y 100 millones de filas:

<img src="Img/Articulo_2_diag_13.png">

¡Parece que en cada operación, 'Pandas' es más lento que 'NumPy'!

La situación (como era de esperar) no cambia cuando aumenta el número de columnas. En cuanto al número de filas, la dependencia (en escala logarítmica) se ve así:

<img src="Img/Articulo_2_diag_14.png">

Pandas parece ser 30 veces más lento que NumPy para matrices pequeñas (menos de cien filas) y tres veces más lento para matrices grandes (más de un millón de filas).

¿Cómo puede ser? Tal vez sea hora de enviar una solicitud de función para sugerir la reimplementación de Pandas a df.column.values.sum() través de 'df.column.sum()'? La propiedad values aquí proporciona acceso a la matriz 'NumPy' subyacente y da como resultado una aceleración de 3 a 30 veces.

La respuesta es no. Pandas es muy lento en esas operaciones básicas porque maneja correctamente los valores faltantes. Pandas necesita NaN (no-un-número) para toda esta maquinaria similar a una base de datos, como la agrupación y el pivoteo, además de que es algo común en el mundo real. En Pandas, se ha trabajado mucho para unificar el uso de 'NaN' en todos los tipos de datos admitidos. Por definición (impuesta a nivel de CPU), nan + cualquier cosa da como resultado nan. Entonces:

<img src="Img/Articulo_2_diag_15.png">

pero

<img src="Img/Articulo_2_diag_16.png">

Una comparación justa sería utilizar 'sum' 'np.nan' en lugar de np.sum, 'mean' 'np.nan' en lugar de 'np.mean' y así sucesivamente. Y de repente…

<img src="Img/Articulo_2_diag_17.png">

'Pandas' es 1,5 veces más rápido que 'NumPy' para matrices con más de un millón de elementos. Sigue siendo 15 veces más lento que 'NumPy' para matrices más pequeñas, pero por lo general no importa mucho si la operación se completa en '0,5 ms' o '0,05 ms': de todos modos es rápido.

En definitiva, si estás 100 % seguro de que no faltan valores en tus columnas, tiene sentido usar 'df.column.values.sum()' en lugar de 'df.column.sum()' para tener un aumento de rendimiento de 'x3' a 'x30'. En presencia de valores faltantes, la velocidad de Pandas es bastante decente e incluso supera a 'NumPy' para matrices enormes (más de 10⁶ elementos).

## Parte 2. Series e índices

<img src="Img/Articulo_2_diag_18.png">

'Series' es una contraparte de una matriz unidimensional en 'NumPy' y es un componente básico para un 'DataFrame' que representa su columna. Aunque su importancia práctica está disminuyendo en comparación con un 'DataFrame' (puede resolver perfectamente muchos problemas prácticos sin saber qué es una 'serie'), es posible que le resulte difícil comprender cómo funcionan los 'DataFrames' sin aprender primero sobre 'series' e 'índices'.

Internamente, 'Series' almacena los valores en un simple vector 'NumPy'. Como tal, hereda sus ventajas (diseño compacto de la memoria, acceso aleatorio rápido) y desventajas (homogeneidad de tipos, eliminaciones e inserciones lentas). Además, Series permite acceder a sus valores por etiqueta utilizando una estructura similar a un diccionario llamada 'index'. Las etiquetas pueden ser de cualquier tipo (comúnmente cadenas y marcas de tiempo). No necesitan ser únicas, pero se requiere la unicidad para aumentar la velocidad de búsqueda y se asume en muchas operaciones.

<img src="Img/Articulo_2_diag_19.png">

Como puedes ver, ahora cada elemento puede ser direccionado de dos maneras alternativas: por 'etiqueta' (usando el índice) y por 'posición' (sin usar el índice):

<img src="Img/Articulo_2_diag_20.png">

A veces, al direccionamiento "por posición" se le denomina "por índice posicional", lo que no hace más que aumentar la confusión.

Obviamente, un par de corchetes no es suficiente para esto. En particular:

- s[2:3] No es la forma más conveniente de abordar el elemento 'número 2'

- Si las etiquetas son números enteros, 's[1:3]' se vuelve ambiguo. Puede significar etiquetas '1' a '3' inclusive o índices posicionales '1' a '3' excluidos.

Para solucionar estos problemas, Pandas tiene dos "tipos" más de corchetes:

<img src="Img/Articulo_2_diag_21.png">

- .loc[] siempre utiliza 'etiquetas' e incluye ambos extremos del intervalo;

- .iloc[] siempre utiliza 'índices' posicionales y excluye el extremo derecho.

El propósito de usar corchetes en lugar de paréntesis aquí es obtener acceso a la conveniente segmentación de 'Python': puede usar dos puntos simples o dobles con el significado familiar de 'start:stop:step'. Como es habitual, falta de inicio ('stop') significa desde el inicio (hasta el 'stop') de la serie. El argumento de 'step' permite hacer referencia a filas pares con s.iloc[::2]y obtener elementos en orden inverso cons['Paris':'Oslo':-1]

También admiten la indexación booleana (indexación con una matriz de booleanos), como muestra esta imagen:

<img src="Img/Articulo_2_diag_22.png">

Y puedes ver cómo admiten la 'indexación elegante' (indexación con una matriz de números enteros) en esta imagen:

<img src="Img/Articulo_2_diag_23.png">

Lo peor de 'Series' es su representación visual: por alguna razón, no recibió una agradable apariencia de texto enriquecido, por lo que parece un ciudadano de 'segunda clase' en comparación con un 'DataFrame':

<img src="Img/Articulo_2_diag_24.png">

He 'parcheado' la serie para que se vea mejor, como se muestra a continuación:

<img src="Img/Articulo_2_diag_25.png">

In [9]:
import numpy as np
import pandas as pd
import pdi

pdi.patch_series_repr(footer=False)

pd.Series(['cat', 'dog', 'panda', 'cat', 'dragon'], index= list('abcde'), name='animal')

,animal
a,cat
b,dog
c,panda
d,cat
e,dragon


La 'línea vertical' significa que se trata de una serie, no de un 'marco de datos'. El pie de página está deshabilitado aquí, pero puede ser útil para mostrar tipos de datos, especialmente con elementos categóricos.

También puedes mostrar varias 'Series' o 'DataFrames' uno al lado del otro con 'pdi'.sidebyside(obj1, obj2, …):

<img src="Img/Articulo_2_diag_26.png">



pdi(significa 'p' y como se 'ilustra' ) es una biblioteca de código abierto en 'github'' con esta y otras funciones para este artículo. Para usarla, escribe

'pip install pandas-illustrated'

#### Índice

El objeto responsable de obtener los elementos de la serie (así como las filas y columnas del DataFrame) por etiqueta se denomina 'índice'. Es rápido: puede obtener el resultado en tiempo constante, ya sea que tenga cinco elementos o 5 mil millones de elementos.

'Indexes' una criatura verdaderamente polimórfica. De manera predeterminada, cuando crea una serie (o un DataFrame) sin argumentos 'index', se inicializa en un objeto perezoso similar al 'range()' de Python . Al igual que 'range()', apenas utiliza memoria y proporciona las etiquetas que coinciden con la indexación posicional. Creemos una serie de un millón de elementos:


In [20]:
s= pd.Series(np.zeros(10**6))
s.index

RangeIndex(start=0, stop=1000000, step=1)

In [34]:
import pandas as pd

indice = pd.RangeIndex(start=0, stop= 1000000, step= 1)
print(f"Memoria usada en 'bytes': {indice.memory_usage()}") # Número de bytes
                                                            # Lo mismo que para Series([0.]) 

Memoria usada en 'bytes': 132


Ahora, si eliminamos un elemento, el índice se transforma implícitamente en una estructura similar a un diccionario, de la siguiente manera:

In [41]:
import pandas as pd

s1= indice.drop(1) 
s1

Index([     0,      2,      3,      4,      5,      6,      7,      8,      9,
           10,
       ...
       999990, 999991, 999992, 999993, 999994, 999995, 999996, 999997, 999998,
       999999],
      dtype='int64', length=999999)

¡Esta estructura consume 8 MB de memoria! Para deshacerse de ella y volver a la estructura liviana de tipo rango, escriba:

In [47]:
import pandas as pd

s2= pd.Series(s1).reset_index(drop=True)
print(s2)
print()
s2 = pd.RangeIndex(start=0, stop= 999999, step= 1)
print(indice)
print()
print(f"Memoria usada en 'bytes': {s2.memory_usage()}")


0              0
1              2
2              3
3              4
4              5
           ...  
999994    999995
999995    999996
999996    999997
999997    999998
999998    999999
Length: 999999, dtype: int64

RangeIndex(start=0, stop=999999, step=1)

Memoria usada en 'bytes': 132


Si no conoce Pandas, es posible que se pregunte por qué Pandas no lo hizo por sí solo. Bueno, para las etiquetas no numéricas, es bastante obvio: ¿por qué (y cómo) Pandas, después de eliminar una fila, volvería a etiquetar todas las filas subsiguientes? Para las etiquetas numéricas, la respuesta es un poco más complicada.

Primero, como ya hemos visto, Pandas te permite referenciar filas puramente por posición, por lo que si quieres direccionar la fila número 5 después de eliminar la fila número 3, puedes hacerlo sin reindexar (para eso está iloc).

En segundo lugar, mantener las etiquetas originales es una forma de mantener una conexión con un momento del pasado, como un botón para "guardar el juego". Imagina que tienes una tabla grande con cien columnas y un millón de filas y necesitas encontrar algunos datos. Estás haciendo varias consultas una por una, cada vez acotando la búsqueda, pero mirando solo un subconjunto de las columnas, porque no es práctico ver todos los cien campos al mismo tiempo. Ahora que has encontrado las filas de interés, quieres ver toda la información sobre ellas en la tabla original. Un índice numérico te ayuda a obtenerla inmediatamente sin ningún esfuerzo adicional. Esquemáticamente, se ve así:

<img src="Img/Articulo_2_diag_27.png">

En general, es una buena idea mantener los valores de índice únicos. Por ejemplo, no obtendrá un aumento en la velocidad de búsqueda en presencia de valores duplicados en el índice. Pandas no tiene una "restricción única" como las bases de datos relacionales ( la característica aún es experimental), pero tiene funciones para verificar si los valores en el índice son únicos y para deshacerse de los duplicados de varias maneras.

A veces, una sola columna no es suficiente para identificar de forma única la fila. Por ejemplo, a veces sucede que las ciudades con el mismo nombre se encuentran en diferentes países o incluso en diferentes regiones del mismo país. Por lo tanto, (city, state) es un mejor candidato para identificar un lugar que una sola 'city'. En las bases de datos, se denomina "clave principal compuesta". En Pandas, se denomina 'MultiIndex' (consulte la Parte 4 a continuación) y cada columna dentro del índice se denomina "nivel".

Otra cualidad sustancial de un índice es que es inmutable. A diferencia de las columnas ordinarias en el 'DataFrame', no se puede modificar en el lugar. Cualquier cambio en el índice implica obtener datos del índice anterior, modificarlos y volver a adjuntar los nuevos datos como un nuevo índice. Por ejemplo, para convertir nombres de columnas en cadenas en el lugar (ahorra memoria), escribir 'df.columns = df.columns.astype(str)' o no en el lugar (útil para encadenar métodos). La mayoría de las veces, esto sucede de forma transparente (por ejemplo, al agregar o eliminar una columna), pero es la inmutabilidad la que no le permite simplemente escribir, por lo que debe recurrir a un método menos obvio '.df.set_axis(df.columns.astype(str), axis=1)' "df.City.name = 'city'" "df.rename(columns={'City': 'city'})"

El índice tiene un nombre (en el caso de MultiIndex, cada nivel tiene un nombre). Lamentablemente, este nombre no se usa lo suficiente en Pandas. Una vez que haya incluido la columna en el índice, 'df.column_name' ya no podrá utilizar la notación conveniente y deberá volver a la menos legible df.indexo a la más universal 'df.loc[]'. La situación empeora con 'MultiIndex'. Una excepción destacada es 'df.merge' que puede especificar la columna que se fusionará por nombre, sin importar si esta columna pertenece al índice o no.

Las columnas se etiquetan utilizando exactamente el mismo índice que las filas, aunque esto puede no ser evidente a partir de los argumentos del 'pd.DataFrameconstructor'.


#### Encontrar elemento por valor

Considere el siguiente objeto 'Serie':

<img src="Img/Articulo_2_diag_28.png">

'Index' ofrece una forma rápida y cómoda de buscar un valor por etiqueta. Pero ¿qué ocurre con la búsqueda de una etiqueta por valor?

<img src="Img/Articulo_2_diag_29.png">

He escrito un par de instrucciones llamados find() y findall() que son rápidos (ya que eligen automáticamente el comando real en función del tamaño de la serie) y más fáciles de usar. Así es como se ve el código:

In [ ]:
>>> import pdi
>>> pdi.find(s, 2)

'penguin'

>>> pdi.findall(s, 4)

Index(['cat', 'dog'], dtype= "object")

#### Valores faltantes

Los desarrolladores de Pandas han tenido especial cuidado con los valores faltantes. Normalmente, se recibe un dataframe con 'NaN' proporcionando una bandera a 'read_csv'. De lo contrario, se puede utilizar 'None' en el constructor o en un operador de asignación (funcionará a pesar de que se implementa de forma ligeramente diferente para diferentes tipos de datos), por ejemplo:

<img src="Img/Articulo_2_diag_31.png">

Lo primero que puedes hacer con los 'NaN' es saber si tienes alguno. Como se ve en la imagen de arriba, 'isna()' genera una matriz booleana y proporciona la cantidad total de valores faltantes..'sum()'

Ahora que sabes que están ahí, puedes optar por deshacerte de ellos eliminándolos, rellenándolos con un valor constante o interpolándolos, como se muestra a continuación:

<img src="Img/Articulo_2_diag_32.png">

Por otro lado, puedes seguir usándolos. La mayoría de las funciones de Pandas ignoran sin problemas los valores faltantes:

<img src="Img/Articulo_2_diag_33.png">

Las funciones más avanzadas ( median, rank, quantile, etc.) también lo hacen.

Las operaciones aritméticas se alinean con 'index':

<img src="Img/Articulo_2_diag_34.png">

Los resultados son inconsistentes en presencia de valores no únicos en el índice. No utilice operaciones aritméticas en series con un índice no único.

#### Comparaciones

Comparar matrices con valores faltantes es complicado.

Para empezar,
- Nonesiempre es igual a None,

- NaN( np.nantambién conocido como math.nanaka float('nan')) nunca es igual a NaN, y si todavía no es lo suficientemente extraño para ti...

- Al comparar pd.NAcon cualquier cosa siempre devuelve pd.NA:

In [ ]:
>>> None == None

True

>>> np.nan == np.nan

False

>>> np.NA == np.NA

<NA>

Cuando se trata de matrices, la cosa empeora:

In [ ]:
>>> np.all(pd.Series([1, None, 3.]) == pd.Series([1., None, 3.])) # Esto es 'np.nan'

False

>>> np.all(pd.Series(['a', None, 'c']) == pd.Series(['a', None, 'c'])) # Esto es 'None'

False

np.all(pd.Series([1, None, 3], dtype='Int64') == pd.Series([1, None, 3], dtype='Int64')) # Esto es pd.NA

True

Un método sencillo para compararlos correctamente es reemplazar todos estos tipos de pd.NA(que significa 'no disponible') con algo que seguramente no estará en la matriz. Por ejemplo, con '', -1 o ∞:

In [ ]:
>>> np.all(s1.fillna(np.inf) == s2.fillna(np.inf)) # Funciona para todos los tipos de datos

True

Una mejor manera es utilizar una función de comparación estándar de 'NumPy' o 'Pandas':

In [ ]:
>>> s = pd.Series([1., None, 3.]) 
>>> np.allclose(s.values, s.values, equal_nan=True) 

True 

>>> len(s.compare(s)) == 0 

True 

>>> pd.testing.assert_series_equal(s, s) # devuelve None

Aquí, 'compare' devuelve una lista de diferencias (un DataFrame, en realidad), 'allclose' devuelve 'array_equal' un valor booleano directamente y 'assert_series_equal' genera una excepción detallada cuando se encuentran diferencias.

Al comparar 'DataFrames' con tipos que no son "nativos" de 'NumPy' (por ejemplo 'object', 'Int64', , tipos mixtos, etc.), la comparación de 'NumPy' falla ( problema n.° 19205 ), mientras que 'Pandas' funciona perfectamente bien. Así es como se ve:

In [ ]:
>>> df = pd.DataFrame({'a': [1., Ninguno, 3.], 'b': ['x', Ninguno, 'z']}) 
>>> np.allclose(df.values, df.values, equal_nan=True) 

TypeError 

<...> 
>>> len(df.compare(df)) == 0 

True 

>>> pd.testing.assert_frame_equal(df, df) # devuelve Ninguno

Una ventaja agradable de usar funciones es su capacidad de comparar números flotantes correctamente (con tolerancias absolutas y relativas personalizables), tal como , de modo que dentro de las matrices 0,1+0,2 parece ser igual a 0,3: 'assert_anything_equal' 'np.allclose'

In [ ]:
>>> 0.1+0.2 == 0.3
 
Falso 

>>> assert_series_equal(pd.Series([0.1+0.2]), pd.Series([0.3])) # sin excepción

Tenga en cuenta que todos los métodos descritos anteriormente requieren que la forma (NumPy) o tanto la forma como el índice (Pandas) sean idénticos. De lo contrario,
los operadores de comparación 'compare' llegan hasta el punto de generar un 'ValueError' mientras 'np.array_equal' retorna 'False' y genera un con 'deltas' como de costumbre. La única excepción es que transmite las matrices antes de la comparación: 'assert_anything_equal' 'AssertionError' 'np.allclose'

In [ ]:
>>> np.array_equal(np.array([1, 1, 1]), np.array[1]) 

Falso 

>>> np.allclose(np.array([1, 1, 1]), np.array[1])
 
Verdadero 

>>> np.testing.assert_array_equal(np.array([1, 1, 1]), np.array[1]) 

Error de afirmación

#### Anexiones, inserciones, eliminaciones

Aunque se supone que los objetos Series son inmutables en tamaño, es posible agregar, insertar y eliminar elementos en el lugar, pero todas esas operaciones son:

- Lento, ya que requieren reasignar memoria para todo el objeto y actualizar el índice;

- dolorosamente inconveniente

Aquí hay una forma de insertar un valor y dos formas de eliminar los valores:

<img src="Img/Articulo_2_diag_35.png">

El segundo método para eliminar valores (a través de drop) es más lento y puede generar errores complejos en presencia de valores no únicos en el índice.

Pandas tiene el método, pero solo puede insertar columnas (no filas) en un marco de datos (y no funciona en absoluto con series).df.insert

Otro método para agregar e insertar es cortar el DataFrame con iloc, aplicar las conversiones necesarias y luego volver a colocarlo con concat. He implementado una función llamada insertque automatiza el proceso:

<img src="Img/Articulo_2_diagram_36.png">

Tenga en cuenta que (al igual que en df.insert) el lugar a insertar, viene dado por una posición 0<=i<=len(s), no por la etiqueta del elemento del índice.

Puede proporcionar una etiqueta para un nuevo elemento. Para un índice no numérico, es obligatorio. Por ejemplo:

<img src="Img/Articulo_2_diag_37.png">

Para especificar el punto de inserción por etiqueta, puede combinar 'pdi.find' con 'pdi.insert', como se muestra a continuación:

<img src="Img/Articulo_2_diag_38.png">

Tenga en cuenta que, a diferencia de 'df.insert', 'pdi.insert' devuelve una copia en lugar de modificar la 'Serie/DataFrame' en el lugar.

#### Estadistica

Pandas ofrece un espectro completo de funciones estadísticas que pueden brindarle una idea de lo que hay en una serie o un marco de datos de un millón de elementos sin tener que desplazarse manualmente por los datos.

Todas las funciones estadísticas de Pandas ignoran los 'NaN', como puedes ver a continuación:

<img src="Img/Articulo_2_diag_39.png">

Tenga en cuenta que 'Pandas std' ofrece resultados diferentes a los de 'NumPy.std'.

Dado que se puede acceder a cada elemento de una serie mediante una etiqueta o un índice posicional, existe una función hermana para argmin (argmax) llamada idxmin ( idxmax), que se muestra en la imagen:

<img src="Img/Articulo_2_diag_40.png">

A continuación se muestra una lista de funciones estadísticas autodescriptivas de Pandas como referencia:

- std, desviación estándar de la muestra;

- var, varianza imparcial;

- sem, error estándar no sesgado de la media;

- quantile, cuantil de muestra ( s.quantile(0.5) ≈ s.median());

- mode, el(los) valor(es) que aparecen con más frecuencia;

- nlargest y nsmallest, por defecto, en orden de aparición;

- diff, primera diferencia discreta;

- cumsum y cumprod, suma acumulada, y producto;

- cummin y cummax, mínimo y máximo acumulativo.

Y algunas funciones estadísticas más especializadas:

- pct_change, cambio porcentual entre el elemento actual y el anterior;

- skew, asimetría imparcial (tercer momento);

- kurt o bien kurtosis, curtosis imparcial (cuarto momento);

- cov, corr y autocorr, covarianza, correlación y autocorrelación;

- rolling, weighted, y ventanas exponentially weighted.

#### Datos duplicados

Se tiene especial cuidado en detectar y tratar los datos duplicados, como se puede ver en la imagen:

<img src="Img/Articulo_2_diag_41.png">

'drop_duplicates' y 'duplicated' puede mantener la última ocurrencia en lugar de la primera.

De hecho, estas dos funciones son complementarias entre sí:
df.drop_duplicates() == df[~df.duplicated()]

Tenga en cuenta que 's.unique()' es más rápido que 'np.unique' (O(N) vs O(NlogN)) y conserva el orden en lugar de devolver los resultados ordenados como np.unique lo hace.

Los valores faltantes se tratan como valores ordinarios, lo que a veces puede generar resultados sorprendentes.

<img src="img/Articulo_2_diag_42.png">



Si desea excluir los 'NaN', debe hacerlo explícitamente. En este ejemplo en particular, 's.dropna().is_unique == True'.

También existe una familia de propiedades monótonas con nombres autodescriptivos:

- s.is_monotonic_increasing,

- s.is_monotonic_decreasing, y, de manera bastante inesperada,

- s.is_monotonic — que es un sinónimo 's.is_monotonic_increasing()' y devuelve 'False' una 
serie decreciente monótona (afortunadamente, obsoleto y eliminado en Pandas 2.0)

No hay funciones documentadas que verifiquen la monotonía estricta, pero puedes construir una fácilmente combinándolas con 's.unique', por ejemplo, para verificar si saumenta estrictamente de manera monótona, escribe
's.unique' y 's.is_monotonic_increasing'

#### Cadenas y expresiones regulares

Prácticamente todos los métodos de cadena de Python tienen una versión vectorizada en Pandas:

<img src="Img/Articulo_2_diag_43.png">

Cuando una operación de este tipo devuelve varios valores, tienes varias opciones sobre cómo utilizarlos:

<img src="Img/Articulo_2_diag_44.png">

Si conoce expresiones regulares, Pandas también tiene versiones vectorizadas de las operaciones comunes con ellas:

<img src="Img/Articulo_2_diag_45.png">

#### Agrupar por

Una operación común en el procesamiento de datos es calcular algunas estadísticas no sobre todo el conjunto de datos sino sobre ciertos grupos de ellos. El primer paso es construir un objeto perezoso proporcionando criterios para dividir una serie (o un marco de datos) en grupos. Este objeto perezoso no tiene una representación significativa, pero puede ser:

- iterado (produce la clave de agrupación y la subserie correspondiente, ideal para depuración):

<img src="Img/Articulo_2_diag_46.png">

- Se consulta de la misma manera que las series ordinarias para obtener una determinada propiedad de cada grupo (es más rápido que la iteración):

<img src="Img/Articulo_2_diag_47.png">

En este ejemplo, dividimos la serie en tres grupos según la parte entera de dividir los valores por 10. Para cada grupo, solicitamos la suma de los elementos, la cantidad de elementos y el valor promedio en cada grupo.

Además de esas funciones de agregación, puedes acceder a elementos específicos en función de su posición o valor relativo dentro de un grupo. Así es como se ve:

<img src="Img/Articulo_2_diag_48.png">

También puede calcular varias funciones en una sola llamada con g.agg(['min', 'max'])o mostrar un montón de funciones estadísticas a la vez con 'g.describe()'.

Si esto no es suficiente, también puedes pasar los datos a través de tu propia función de Python. Puede ser:

- Una función 'f' que acepta un grupo x(un objeto Serie) y genera un único valor (por ejemplo, sum()) cong.apply(f)

- Una función 'f' que acepta un grupo x(un objeto Serie) y genera un objeto Serie del mismo tamaño que x(por ejemplo, cumsum()) cong.transform(f)

<img src="Img/Articulo_2_diag_49.png">

En los ejemplos anteriores, los datos de entrada están ordenados. Esto no es necesario para 'groupby'. En realidad, funciona igual de bien si los elementos del grupo no se almacenan de forma consecutiva, por lo que es más cercano a 'collections.defaultdict' que a 'itertools.groupby'. Y siempre devuelve un índice sin duplicados.

<Img src="Img/Articulo_2_diag_50.png">

A diferencia de 'defaultdict' la cláusula 'GROUP BY' de una base de datos relacional, Pandas 'groupby' ordena los resultados por nombre de grupo. Puede desactivarse con 'sort='False', como verá en el código:

In [ ]:
>>> s = pd.Series([1, 3, 20, 2, 10])
>>> for k, v in s.groupby(s//10, sort=False):
       print(k, v.tolist())
0 [1, 3, 2]
2 [20]
1 [10]

Descargo de responsabilidad: En realidad, 'g.apply(f)' es más versátil de lo descrito anteriormente:

- Si f(x) devuelve una serie del mismo tamaño que x, puede imitar la transformación

- Si f(x) devuelve una serie de diferente tamaño o un marco de datos, da como resultado una serie con un Multiindex correspondiente.

Pero los documentos advierten que esos usos pueden ser más lentos que los métodos correspondientes transform, aggasí que tenga cuidado.

## Parte 3. DataFrame

<img src="Img/Articulo_2_diag_51.png">

La estructura de datos principal de Pandas es un 'DataFrame'. Agrupa una matriz bidimensional con etiquetas para sus filas y columnas. Consta de una serie de objetos Series (con un índice compartido), cada uno de los cuales representa una sola columna y posiblemente tenga diferentes tipos de datos.

#### Lectura y escritura de archivos CSV

Una forma común de construir un 'DataFrame' es leyendo un archivo CSV (valores separados por comas), como muestra esta imagen:

<img src="Img/Articulo_2_diag_52.png">

La función es una herramienta totalmente automatizada y muy personalizable. Si quieres aprender una sola cosa sobre Pandas, aprende a usarlo. ¡ Valdrá la pena!:) 'pd.read_csv()read_csv'

A continuación se muestra un ejemplo de análisis de un archivo CSV no estándar:

<img src="Img/Articulo_2_diag_53.png">

Y una breve descripción de algunos de los argumentos:

<img src="Img/Articulo_2_diag_54.png">

Dado que CSV no tiene una especificación estricta, a veces es necesario un poco de ensayo y error para leerlo correctamente. Lo bueno 'read_csv' es que detecta automáticamente muchas cosas, entre ellas:

- Nombres y tipos de columnas,

- Representación de booleanos,

- Representación de valores faltantes, etc.

Al igual que con cualquier automatización, es mejor asegurarse de que haya hecho lo correcto. Si los resultados de escribir simplemente 'df' en una celda de Jupyter resultan ser demasiado largos (o demasiado incompletos), puede probar lo siguiente:

- df.head(5) o df[:5] muestra las primeras cinco filas

- df.dtypes devuelve los tipos de columna

- df.shape devuelve el número de filas y columnas

- df.info() Resume toda la información relevante

Es una buena idea establecer una o varias columnas como índice. La siguiente imagen muestra este proceso:

<img src="Img/Articulo_2_diag_55.png">



Index tiene muchos usos en Pandas:

- Hace que las búsquedas por columnas indexadas sean más rápidas;

- Las operaciones aritméticas, el apilamiento y la unión se alinean por índice, etc.

Todo esto se produce a costa de un consumo de memoria algo mayor y una sintaxis un poco menos obvia.

#### Construyendo un DataFrame

Otra opción es construir un marco de datos a partir de datos ya almacenados en la memoria. Su constructor es tan extraordinariamente omnívoro que puede convertir (o encapsular) cualquier tipo de datos que le introduzcas:

<img src="Img/Articulo_2_diag_56.png">

En el primer caso, en ausencia de etiquetas de fila, Pandas etiquetó las filas con números enteros consecutivos. En el segundo caso, hizo lo mismo con las filas y las columnas. Siempre es una buena idea proporcionar a Pandas los nombres de las columnas en lugar de las etiquetas de números enteros (usando el columnsargumento) y, a veces, los nombres de las filas (usando el indexargumento, aunque rowspuede sonar más intuitivo). Esta imagen te ayudará:

<img src="Img/Articulo_2_diag_57.png">

Para asignar un nombre a la columna de índice, escriba df.index.name = 'city_name' o usepd.DataFrame(..., index=pd.Index(['Oslo', 'Vienna', 'Tokyo'], name='city_name')).

La siguiente opción es construir un DataFrame a partir de un diccionario de vectores 'NumPy' o una matriz NumPy 2D:

<img src="Img/Articulo_2_diag_58.png">

Observe cómo los population valores se convirtieron en flotantes en el segundo caso. En realidad, esto ocurrió antes, durante la construcción de la matriz 'NumPy'. Otra cosa que debe tener en cuenta aquí es que la construcción de un marco de datos a partir de una matriz 'NumPy' 2D es una vista predeterminada. Eso significa que al cambiar los valores en la matriz original se cambia el marco de datos y viceversa. Además, ahorra memoria.

Este modo también se puede habilitar en el primer caso (un diccionario de vectores NumPy) configurando copy=False. Sin embargo, es muy frágil. Operaciones simples pueden convertirlo en una copia sin previo aviso.

Dos opciones más (menos útiles) para crear un DataFrame son:

- De una lista de diccionarios (donde cada diccionario representa una sola fila, sus claves son nombres de columnas y sus valores son los valores de celda correspondientes)

- De un 'dict' de Series (donde cada Serie representa una columna; devuelve una copia por defecto, se le puede indicar que devuelva una vista con copy=False).

Si registra datos de transmisión "sobre la marcha", su mejor opción es utilizar un diccionario de listas o una lista de listas porque Python preasigna espacio de manera transparente al final de una lista para que las adiciones sean rápidas. Ni las matrices de 'NumPy' ni los DataFrame de Pandas lo hacen. Otra posibilidad (si conoce la cantidad de filas de antemano) es preasigna memoria manualmente con algo como DataFrame(np.zeros).

#### Operaciones básicas con DataFrames

Lo mejor de DataFrame (en mi opinión) es que puedes:

- Acceder fácilmente a sus columnas, por ejemplo, df.areadevuelve valores de columna (o alternativamente, df[‘area’] — bueno para nombres de columnas que contienen espacios)

- Operar las columnas como si fueran variables independientes, por ejemplo, después de df.population /= 10**6 almacenar la población en millones, y el siguiente comando crea una nueva columna llamada 'densidad' calculada a partir de los valores de las columnas existentes:

<img src="Img/Articulo_2_diag_59.png">

Tenga en cuenta que al crear una nueva columna, los corchetes son obligatorios incluso si su nombre no contiene espacios.

Además, puedes utilizar operaciones aritméticas en columnas incluso de diferentes DataFrames, siempre que sus filas tengan etiquetas significativas, como se muestra a continuación:

<img src="Img/Articulo_2_diag_60.png">

#### Indexación de DataFrames

Como ya hemos visto en la sección Series, los corchetes comunes simplemente no son suficientes para satisfacer todas las necesidades de indexación. No se puede acceder a las filas por etiquetas, no se puede acceder a las filas disjuntas por índice posicional y ni siquiera se puede hacer referencia a una sola celda, ya que df['x', 'y'] está reservada para MultiIndex.

<img src="Img/Articulo_2_diag_61.png">

Para satisfacer esas necesidades, los DataFrames, al igual que las series, tienen dos modos de indexación alternativos: 'loc' para indexar por etiquetas y 'iloc' para indexar por índice posicional.

<img src="Img/Articulo_2_diag_62.png">

En Pandas, hacer referencia a varias filas o columnas es una copia, no una vista. Pero es un tipo especial de copia que permite realizar asignaciones en su totalidad:

- df.loc['a']=10 Trabajos (una sola fila se puede escribir como un todo)

- df.loc['a']['A']=10 Trabajos (el acceso al elemento se propaga al original df)

- df.loc['a':'b'] = 10 Trabajos (asignar a un subarreglo como un trabajo completa)

- df.loc['a':'b']['A'] = 10 no (asignar a sus elementos no lo hace).

En el último caso, el valor solo se establecerá en una copia de una porción y no se reflejará en el original df(se mostrará una advertencia en consecuencia).

Dependiendo de la situación existen diferentes soluciones:

1. Quiere cambiar el marco de datos original df. Entonces use
df.loc['a':'b', 'A'] = 10

2. Ha realizado la copia intencionalmente y desea trabajar en esa copia:
df1 = df.loc['a':'b']; df1['A']=10 # SettingWithCopy warning para deshacerse de una advertencia en esta situación, conviértala en una copia real:

df1 = df.loc['a':'b'].copy(); df1['A']=10

Pandas también admite una sintaxis 'NumPy' conveniente para la indexación booleana.

<img src="Img/Articulo_2_diag_63.png">

Al utilizar varias condiciones, estas deben estar entre paréntesis, como puedes ver a continuación:

<img src="Img/Articulo_2_diag_64.png">

Cuando esperas que se devuelva un único valor, necesitas tener especial cuidado.

<img src="Img/Articulo_2_diag_65.png">


Dado que potencialmente podría haber varias filas que coincidan con la condición, 'loc' se devolvió una serie. Para obtener un valor escalar, puede utilizar:

- float(s) o una más universal s.item()que generará ValueError a menos que haya exactamente un valor en la serie

- s.iloc[0] que solo lanzará una excepción cuando no se encuentre nada; además, es el único que admite asignaciones: df[…].iloc[0] = 100, pero seguramente no lo necesitas cuando quieres modificar todas las coincidencias: df[…] = 100.

Alternativamente, puede utilizar consultas basadas en cadenas:

- df.query('name=="Vienna"')

- df.query('population>1e6 and area<1000')

Son más cortos, funcionan muy bien con 'MultiIndex' y los operadores lógicos tienen prioridad sobre los operadores de comparación (se requieren menos paréntesis), pero solo pueden filtrar por filas y no se puede modificar el 'DataFrame' a través de ellos.

Varias bibliotecas de terceros le permiten usar la sintaxis SQL para consultar los DataFrames directamente (duckdb) o indirectamente copiando el dataframe a 'SQLite' y envolviendo los resultados nuevamente en objetos Pandas (pandasql). Como era de esperar, el método directo es más rápido.

#### Aritmética de DataFrame